In [1]:
## import required libraries
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('punkt')
#nltk.download('stopwords')
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import string

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.models import load_model

Using TensorFlow backend.


In [2]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
import pandas as pd
fake_news= pd.read_csv("fake.csv")
fake_news.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true_news= pd.read_csv("True.csv")
true_news.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
# Introducing new column in both dataframes
true_news["label"]=1
fake_news["label"]=0
#concatenating the two dataframes in one single dataframe
all_data=pd.concat([true_news,fake_news],axis=0)

In [6]:
all_data.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [7]:
all_data.sample(10)

,title,text,subject,date,label
10234,Trump warns Republicans against pushing a thir...,WASHINGTON (Reuters) - Republican presidential...,politicsNews,"March 21, 2016",1
4754,Senior House Democrat says Republicans short o...,WASHINGTON (Reuters) - The No. 2 Democrat in t...,politicsNews,"March 23, 2017",1
16270,BREAKING: Black Caucus Chair Threatens Over “C...,A spat between black lawmakers and Republicans...,Government News,"Jan 10, 2017",0
14522,LET’S PLAY! CAN YOU ANSWER The 21 Questions Th...,CAN YOU DO IT? WE D LOVE TO KNOW WHAT SURPRISE...,politics,"Feb 6, 2016",0
1491,Senators urge Trump administration to act on M...,WASHINGTON (Reuters) - A group of Republican a...,politicsNews,"September 28, 2017",1
7915,A Trump loss would 'upset the verdict of histo...,(Corrects this Oct. 5 story’s headline to con...,politicsNews,"October 5, 2016",1
7542,"Amid scandal, bluster and insults, Trump close...",WASHINGTON (Reuters) - Supercharging the blust...,politicsNews,"November 6, 2016",1
16065,China says position on THAAD system in South K...,BEIJING (Reuters) - China s foreign ministry s...,worldnews,"October 31, 2017",1
8504,Clinton extends lead over Trump to 7 points: R...,NEW YORK (Reuters) - Democratic presidential n...,politicsNews,"August 9, 2016",1
4759,"Bin Laden’s Brother Lived In Trump Tower, Gav...","Back in June, on Facebook, Donald Trump called...",News,"September 5, 2016",0


In [8]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
title      44898 non-null object
text       44898 non-null object
subject    44898 non-null object
date       44898 non-null object
label      44898 non-null int64
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [9]:
# Combining title and text to obtain a single string
# dropping title and text columns and add the combined column
all_data['fulltext'] = all_data.title + ' ' + all_data.text
all_data.drop(['title','text'], axis=1, inplace=True)


In [10]:
all_data.sample(10)

,subject,date,label,fulltext
14329,politics,"Mar 10, 2016",0,OBAMA’S DOJ TO BLAME IRAN FOR CYBER-ATTACK On ...
10069,politics,"Aug 22, 2017",0,WATCH: MASSIVE Crowd Of Supporters Line Street...
18592,worldnews,"October 1, 2017",1,"Match North Korea overture with Iran offer, Ge..."
23443,Middle-east,"February 20, 2016",0,ENCRYPTION TRUTH: What The FBI Aren’t Telling ...
2299,News,"March 4, 2017",0,"‘March 4 Trump’ Gets HILARIOUSLY Bad Turnout,..."
10717,politicsNews,"February 23, 2016",1,More Californians register to vote but fewer a...
3702,politicsNews,"May 18, 2017",1,Democratic senator tussles with U.S. Treasury ...
6470,News,"May 8, 2016",0,"For The 7th Year, Every Urban Prep Graduate A..."
19476,worldnews,"September 21, 2017",1,Slovenian government faces test in pre-electio...
8257,politicsNews,"September 6, 2016",1,No. 2 House Democrat opposes temporary gov't s...


In [37]:
## create dataframe containg the combined title, text column and the label column
combined_data= all_data[["fulltext","label"]]
combined_data.head(20)

,fulltext,label
0,"As U.S. budget fight looms, Republicans flip t...",1
1,U.S. military to accept transgender recruits o...,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,1
3,FBI Russia probe helped by Australian diplomat...,1
4,Trump wants Postal Service to charge 'much mor...,1
5,"White House, Congress prepare for talks on spe...",1
6,"Trump says Russia probe will be fair, but time...",1
7,Factbox: Trump on Twitter (Dec 29) - Approval ...,1
8,Trump on Twitter (Dec 28) - Global Warming The...,1
9,Alabama official to certify Senator-elect Jone...,1


In [12]:
combined_data =combined_data.reset_index()
combined_data.drop(['index'], axis=1, inplace=True)

In [36]:
combined_data[combined_data["label"]==1]["fulltext"].values[5]

'White House, Congress prepare for talks on spending, immigration WEST PALM BEACH, Fla./WASHINGTON (Reuters) - The White House said on Friday it was set to kick off talks next week with Republican and Democratic congressional leaders on immigration policy, government spending and other issues that need to be wrapped up early in the new year. The expected flurry of legislative activity comes as Republicans and Democrats begin to set the stage for midterm congressional elections in November. President Donald Trump’s Republican Party is eager to maintain control of Congress while Democrats look for openings to wrest seats away in the Senate and the House of Representatives. On Wednesday, Trump’s budget chief Mick Mulvaney and legislative affairs director Marc Short will meet with Senate Majority Leader Mitch McConnell and House Speaker Paul Ryan - both Republicans - and their Democratic counterparts, Senator Chuck Schumer and Representative Nancy Pelosi, the White House said. That will be

In [14]:
#checking for missing data
combined_data.isnull().sum()

fulltext    0
label       0
dtype: int64

In [15]:
combined_data.shape

(44898, 2)

we can configure that the data does not have null values and the total size of the data 44898 rows and two colunms (full_text,labels)

# Data cleaning/preprocessing 

In [16]:
## data prepartion 
X_data = combined_data['fulltext']
y_data = combined_data.label
X_data = X_data.astype(str)
##function to extract major words from true and fake news
def final_processed(X_data_full):
     # function for removing punctuations
    def remove_punct(X_data_func):
        string1 = X_data_func.lower()
        translation_table = dict.fromkeys(map(ord, string.punctuation),' ')
        string2 = string1.translate(translation_table)
        return string2
    
    X_data_full_clear_punct = []
    for i in range(len(X_data_full)):
        test_data = remove_punct(X_data_full[i])
        X_data_full_clear_punct.append(test_data)
        
    #function for removing stopwords
    def remove_stopwords(X_data_func):
        new_file=open ("stopwords_en.txt","r")
        stop_words=new_file.read().split("\n")
        words=word_tokenize(X_data_func)
        pattern =[word for word in words if not word in stop_words] 
        #tokens_without_sw =" ".join(pattern) 
        string2= " ".join(pattern)        
        return string2 
    X_data_full_clear_stopwords = []
    for i in range(len(X_data_full)):
        test_data = remove_stopwords(X_data_full[i])
        X_data_full_clear_stopwords.append(test_data)
        
    # function for tokenizing
    def tokenize_words(X_data_func):
        words = nltk.word_tokenize(X_data_func)
        return words
    
    X_data_full_tokenized_words = []
    for i in range(len(X_data_full)):
        test_data = tokenize_words(X_data_full[i])
        X_data_full_tokenized_words.append(test_data)
        
      # function for lemmatizing
    lemmatizer = WordNetLemmatizer()
    def lemmatize_words(X_data_func):
        words = lemmatizer.lemmatize(X_data_func)
        return words
    
    X_data_full_lemmatized_words = []
    for i in range(len(X_data_full)):
        test_data = lemmatize_words(X_data_full[i])
        X_data_full_lemmatized_words.append(test_data)
    
    return X_data_full_lemmatized_words


In [17]:
data_Y=final_processed(X_data)

In [26]:
  #feature extraction using bag of words model using CountVectorizer and TfidfVectorizer
import pickle   
cv = CountVectorizer(max_features=1000)
cv.fit(data_Y)
X_data_full_vector = cv.transform(data_Y).toarray()
pickle_out = open("CV.pickle","wb")
pickle.dump(cv, pickle_out)
pickle_out.close()
tfidf = TfidfTransformer()   
tfidf.fit(X_data_full_vector)
X_data_full_tfidf =tfidf.transform(X_data_full_vector).toarray()



In [28]:
pickle_out = open("tfidf.pickle","wb")
pickle.dump(tfidf, pickle_out)
pickle_out.close()

In [27]:
#print(X_data_full_vector[20])
X_data_full_tfidf[20]

array([0.02993671, 0.        , 0.        , 0.15179278, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.03531766,
       0.        , 0.        , 0.04245875, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.02587002,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.1695279 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.03882089, 0.        , 0.        , 0.03165311,
       0.0191035 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.03256187, 0.        ,
       0.08694748, 0.        , 0.        , 0.        , 0.     

# ML Models in order to choose the one with the best accuracy 

In [ ]:
# Preparing training and testing data using train_test_split

X_train, X_test, y_train, y_test = train_test_split(data_X, y_data, test_size=0.25, random_state= 4353)

# Multinominal Niave Bayes

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)
toxic_classification_model = MultinomialNB().fit(X_train, y_train)
pred_test_MNB = toxic_classification_model.predict(X_test)
##model eveluation
acc_MNB = accuracy_score(y_test, pred_test_MNB)
print(acc_MNB)

# Random Forest

In [ ]:
rfc=RandomForestClassifier(n_estimators= 10, random_state=4353)
rcf_classifier=rfc.fit(X_train, y_train)
pred_test_RF = rfc.predict(X_test)
##model eveluation
acc_RF = accuracy_score(y_test, pred_test_RF)
print(acc_RF)

# linear SVM

In [ ]:
 from sklearn.svm import LinearSVC

In [ ]:
svc = LinearSVC()
svc_classifier=svc.fit(X_train, y_train)
pred_test_svc =svc.predict(X_test)
##model eveluation
acc_svc = accuracy_score(y_test, pred_test_svc)
print(acc_svc)

# stochastic Gradient descent

In [ ]:
from sklearn.linear_model import SGDClassifier
SDG = SGDClassifier(loss="hinge", penalty="l2", max_iter=10)
SDG.fit(X_train, y_train)
pred_test_SDG =SDG.predict(X_test)
##model eveluation
acc_SDG = accuracy_score(y_test, pred_test_SDG)
print(acc_SDG)

# KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)
pred_test_knn =knn.predict(X_test)
##model eveluation
acc_knn = accuracy_score(y_test, pred_test_knn)
print(acc_knn)

# pickling the model with best accuracy

In [ ]:
import pickle
#pickle.dump(RF_classifier, open('final_RF_prediction.pickle', 'wb'))
pickle.dump(rfc, open('final_RF_prediction.pickle', 'wb'))
pickle_out.close()